In [ ]:
#Imports of all necessary Python Packages
import sklearn as skl
import numpy as np
import pandas as pd
from time import time
import csv

#Classifiers
from sklearn.svm import SVC

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.decomposition import PCA

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegressionCV

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

#Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split

In [ ]:
man_feats = pd.read_csv('../Data/ManualFeaturesData/TrainingManFeats.csv', index_col=0)


In [ ]:
pos_tags = pd.read_csv('../Data/POStaggedData/TrainingPOSTagged.csv', index_col=0)


In [ ]:
dataframe = pos_tags
dataframe = pd.concat([pos_tags,man_feats], axis=1)
# Uncomment to use biling data
# dataframe = pd.concat([dataframe, biling_samples], axis=0, ignore_index = True)
labels = dataframe['labels'].values
data = dataframe.drop('labels', axis=1).values
print data.shape
print labels.shape

In [ ]:
t0 = time()
# Feature Extraction
pca = PCA(n_components = 800)
data_decomposed = pca.fit_transform(data)
print("done in %0.3fs." % (time() - t0))

In [ ]:
estimators = [('lda', LinearDiscriminantAnalysis()),  ('logReg', LogisticRegressionCV()),
              ('adaB', AdaBoostClassifier(base_estimator=None, learning_rate=1, n_estimators=50))]
classfVC = VotingClassifier(estimators= estimators, voting='hard')

In [ ]:
# Train final Classifier:

classf = classfVC
classf.fit(data,labels)

In [ ]:
# Load Test data - Unnecessary 
test = pd.read_csv("../Data/OficialData/test_blind.txt", sep="\t", header=None, names=['labels','phrases'],
                   lineterminator='\n', quoting=csv.QUOTE_NONE)

In [ ]:
test_man_feats = pd.read_csv('../Data/ManualFeaturesData/test_man.csv', index_col=0)
test_pos_tags = pd.read_csv('../Data/POStaggedData/test_data.csv', index_col=0)

In [ ]:
dataframe = pd.concat([test_pos_tags,test_man_feats], axis=1)
# Uncomment to use biling data
#dataframe = pd.concat([dataframe, biling_samples], axis=0, ignore_index = True)
test_data = dataframe.drop('labels', axis=1).values
print test_data.shape
test_data_decomposed = pca.transform(test_data)
print test_data_decomposed.shape


In [ ]:
testlabs = classf.predict(test_data_decomposed)
test.labels = test['labels'] = testlabs
test.to_csv("test_output.txt", sep="\t", header=False, index=False, quoting=csv.QUOTE_NONE, enconding = 'utf-8')